In [1]:
from entsoe import get_dap_dataloader
import numpy as np
from torch_geometric_temporal.nn.attention.stgcn import STConv
import torch
import torch.nn as nn
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class FullyConnLayer(nn.Module):
    def __init__(self, c):
        super(FullyConnLayer, self).__init__()
        self.conv = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        return self.conv(x)


# adapted from Hao Wei
class OutputLayer(nn.Module):
    def __init__(self, c, T, n):
        super(OutputLayer, self).__init__()
        self.tconv1 = nn.Conv2d(c, c, (T, 1), 1, dilation=1, padding=(0, 0))
        self.ln = nn.LayerNorm([n, c])
        self.tconv2 = nn.Conv2d(c, c, (1, 1), 1, dilation=1, padding=(0, 0))
        self.fc = FullyConnLayer(c)

    def forward(self, x):
        x_t1 = self.tconv1(x)
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        x_t2 = self.tconv2(x_ln)
        return self.fc(x_t2)


class Model(nn.Module):
    def __init__(self, window_size, num_nodes):
        super(Model, self).__init__()
        self.stconv1 = STConv(
            num_nodes=num_nodes,
            in_channels=24,
            hidden_channels=16,
            out_channels=12,
            kernel_size=3,
            K=1,
        )
        self.stconv2 = STConv(
            num_nodes=num_nodes,
            in_channels=12,
            hidden_channels=16,
            out_channels=8,
            kernel_size=3,
            K=1,
        )
        # window_size - 2 * num_layers * (kernel_size - 1) = 24 - 2 * 2 * (3 - 1) = 16
        T = window_size - 2 * 2 * (3 - 1)
        self.output_layer = OutputLayer(8, T, 10)

    def forward(self, x, edge_index, edge_weight):
        x = self.stconv1(x, edge_index, edge_weight)
        x = self.stconv2(x, edge_index, edge_weight)
        x = x.permute(0, 3, 1, 2)
        x = self.output_layer(x)
        x = x.permute(0, 2, 3, 1)
        return x

In [3]:
window_size = 24
dataloader = get_dap_dataloader(window_size, 1)

torch.Size([43729, 10, 24]) torch.Size([43729, 2, 32]) torch.Size([43729, 32, 1]) torch.Size([43729, 32, 1])
torch.Size([43704, 24, 10, 24]) torch.Size([43704, 24, 32, 1]) torch.Size([43704, 24, 2, 32]) torch.Size([43704, 10])


In [4]:
static_edge_index = None
for data in dataloader:
    static_edge_index = data[2][0,0,:,:]
static_edge_index = static_edge_index.to(device)

In [7]:
future = 1
model = Model(window_size, 10)
model.to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criteron_mse = nn.MSELoss()
criteron_mae = nn.L1Loss()
for eopch in range(100):
    epoch_loss_mse = 0
    epoch_loss_mae = 0
    for data in tqdm(dataloader):
        optimizer.zero_grad()
        X, edge_weights, edge_index, y = data
        X = X.to(device)
        edge_weights = edge_weights.to(device)
        edge_index = edge_index.to(device)
        y = y.to(device)
        y_hat = model(X, static_edge_index, edge_weights[0,0,:,:])
        y_hat = y_hat.squeeze(0, 1, 3)
        loss_mse = criteron_mse(y_hat, y)
        loss_mae = criteron_mae(y_hat, y)
        epoch_loss_mse += loss_mse.item()
        epoch_loss_mae += loss_mae.item()
        loss_mse.backward()
        optimizer.step()
    print(f"Epoch {epoch}")
    print(epoch_loss_mse / (len_data - window_size - 1))
    print(epoch_loss_mae / (len_data - window_size - 1))

 12%|█▏        | 105/875 [03:25<25:07,  1.96s/it]


KeyboardInterrupt: 